<a href="https://colab.research.google.com/github/Pugianf/Big_Data_and_Public_Sector_I/blob/main/Aula_2_2021_10_14_.complemento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Importando módulos necessários
import numpy as np
import pandas as pd
import pathlib
from zipfile import ZipFile
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
## Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Lendo os arquivos
# Determinando o Caminho
sCaminho = '/content/drive/MyDrive/'

## Lendo o arquivo zipado
# Leitura feita no R e dados já filtrados para NaNs de renda e para apenas pessoas em idade ativa
http ="https://github.com/claudiomar23/Python_open_class/blob/main/PNADC_042019.zip?raw=true"

sArquivo = f"{sCaminho}PNADC_042019.zip"
with ZipFile(sArquivo) as z:
    df = pd.read_csv(z.open(http))
    print(*z.namelist(),sep="\n")

KeyError: ignored

In [ ]:
## Importante: os valores nulos de renda (VD4020) não são pessoas que ganham 0; são pessoas que NÃO TRABALHAM
# Para retirar essas pessoas da base original, 
df.dropna(subset=['VD4020'], inplace=True)

## Retirando pessoas em idade não-ativa
df = df.loc[(df["V2009"] >= 15) & (df["V2009"] <= 65)]

In [ ]:
## Vendo o tamanho da base (lembrando que ela já está filtrada)
print(f"Linhas: {df.shape[0]}; Colunas: {df.shape[1]}")

In [ ]:
## Filtrando apenas para o Centro-Oeste
df = df.loc[df["UF"].isin([50, 51, 52, 53])]
print(f"Linhas: {df.shape[0]}; Colunas: {df.shape[1]}")

## Preparação da base

In [ ]:
## Criando a coluna de identificação dos domicilios
df['iddom'] = df['UPA'].astype(str) + df['V1008'].astype(str) + df['V1014'].astype(str)
df['idind'] = df['iddom'] + df['V2003'].astype(str)

In [ ]:
## Criando idade e idade ao quadrado
df.rename(columns={"V2009":"idade"}, inplace=True)
df['idadesq'] = df['idade'] ** 2

In [ ]:
## Gênero (dummy feminina, mais especificamente)
df['feminino'] = df['V2007'] - 1

## Alternativamente
# df['feminino'] = df["V2007"].apply(lambda i: 1 if i == 2 else 0)

In [ ]:
## V2010: cor
# Substituindo os números pelo nomes e vendo a quantidade
df["V2010"].replace([1,2,3,4,5,9], ['branca','preta','amarela','parda','indigena',np.nan], inplace=True)
df['V2010'].value_counts(dropna=False)

In [ ]:
## Renomeando cor
df.rename(columns={"V2010":"cor"}, inplace=True)

In [ ]:
## criando as dummies e juntando-as ao dataframe
df = pd.concat([df, pd.get_dummies(df['cor'])], axis=1)

In [ ]:
## Vendo se está tudo ok
df[['cor', 'branca','preta','amarela','parda','indigena']]

In [ ]:
## Grau de Educação
# Substituindo os números pelo nomes e vendo a quantidade
df["VD3004"].replace([1,2,3,4,5,6,7], ['sem_instrucao','fund_incompleto','fund_completo','medio_incompleto','medio_completo','superior_incompleto','superior_completo'], inplace= True)
df['VD3004'].value_counts(normalize=True, dropna=False)*100

In [ ]:
## criando as dummies e juntando-as ao dataframe
df = pd.concat([df, pd.get_dummies(df['VD3004'])], axis = 1)

In [ ]:
## V1022: domicilios rurais
df['rural'] = df['V1022'] - 1

In [ ]:
## Vendo proporções
df['rural'].value_counts(normalize=True, dropna=False)*100

In [ ]:
## VD4001: força de trabalho (mais especificamente, fora dela)
df['VD4001'] = df['VD4001'] - 1

## VD4002: ocupação (mais especificamente, pessoas desocupadas)
df['VD4002'] = df['VD4002'] - 1

In [ ]:
#### dummies de ocupação ######
## tipo de trabalho
# mais agregado
df['VD4008'].replace([1,2,3,4,5,6], ['privado','domestico','publico','empregador','conta_propria','familiar'], inplace=True)
df = pd.concat([df,pd.get_dummies(df['VD4008'])], axis = 1)
df = df.drop(['conta_propria','familiar'], axis = 1)

# desagregado
df["VD4009"].replace([1,2,3,4,5,6,7,8,9,10], ['privado_formal','privado_informal','domestico_formal','domestico_informal','publico_formal','publico_informal','militar','empregador1','conta_propria','familiar'], inplace= True)
df = pd.concat([df,pd.get_dummies(df['VD4009'])], axis = 1)

## setor de ocupação
df['VD4010'] = df['VD4010'].replace([1,2,3,4,5,6,7,8,9,10,11,12], ['agro','industria','construcao','comercio','transporte','aloj_alim','servicos','adm_publica','educ_saude','outros_servicos','servicos_domesticos','ativ_mal_definidas'])
df = pd.concat([df,pd.get_dummies(df['VD4010'])], axis = 1)

In [ ]:
## Renomendo educação
df.rename(columns={"VD3005":"educ","VD3004":"grau_educ"}, inplace=True)

# Análises de Renda

Qual variável de renda usar?

In [ ]:
## Vamos usar renda habitual total controlada por horas, por dois motivos:
# 1. Não há nenhum caso em que a renda habitual seja 0, o que facilita a logaritimização e linearização do modelo
# 2. Menos sensível à sazonalidade (fim de ano costuma ter mais renda efetiva em virtude de bônus e contratações temporárias)

# Renomeando as rendas
nomes_renda = {'VD4016':'renda_hab_prin','VD4017':'renda_efet_prin','VD4019':'renda_hab_tot','VD4020':'renda_efet_tot','VD4031':'horas_hab_tot','VD4032':'horas_efet_prin','VD4035':'horas_efet_tot'}
df.rename(columns = nomes_renda, inplace = True)

In [ ]:
## ATENÇÃO: O PAINEL SÓ CONTÉM DADOS DE RENDIMENTOS DO TRABALHO (sem benefícios previdenciários)
## Vendo estatísticas das diferentes rendas e horas trabalhadas
df[['renda_hab_tot','renda_hab_prin','renda_efet_tot','renda_efet_prin','horas_hab_tot','horas_efet_tot']].describe()

In [ ]:
## Para controlar para a oferta de horas de trabalho (mulheres ofertam menos horas)
# dividimos pelo número de horas trabalhadas, o que é mais um motivo para usar rendas habituais
# (rendas efetivas possuem alguns 0, o que causaria problemas de divisão)

df['renda_hab_hora'] = df['renda_hab_tot']/(df['horas_hab_tot']*4)
df['lsalariohora'] = np.log(df['renda_hab_hora'])

## Médias de renda por gênero

In [ ]:
## Média por gênero
df.groupby('feminino')['renda_hab_tot'].mean()

In [ ]:
from scipy import stats

## Subamostra de homens
vRendaHomens = df['renda_hab_tot'].loc[df['feminino'] == 0]

## Subamostra de mulheres
vRendaMulheres = df['renda_hab_tot'].loc[df['feminino'] == 1]

In [ ]:
## Teste
stats.ttest_ind(vRendaHomens, vRendaMulheres, nan_policy='omit')

## Renda por Raça

In [ ]:
# Média por cor

df.groupby('cor')['renda_hab_tot'].mean()

In [ ]:
## Subamostra de amarelos
vRendaAmarela = df['renda_hab_tot'].loc[df['cor'] == 'amarela']

## Subamostra de branca
vRendaBranca = df['renda_hab_tot'].loc[df['cor'] == 'branca']

## Subamostra de indígena
vRendaBranca = df['renda_hab_tot'].loc[df['cor'] == 'indígena']

## Subamostra de parda
vRendaBranca = df['renda_hab_tot'].loc[df['cor'] == 'parda']

## Subamostra de preta
vRendaBranca = df['renda_hab_tot'].loc[df['cor'] == 'preta']

## Vimos que há uma clara disparidade de gênero e raça no salário, mas como medir isso mais precisamente?

In [ ]:
## Importando o módulo e definindo uma função
from statsmodels.formula.api import ols

In [ ]:
## Criando fórmula
formula = "lsalariohora ~ feminino"

## Modelo
mod = ols(formula, df).fit(use_t=True)

## Vendo os resultados
print(mod.summary())

In [ ]:
## Modelo mais completo com educ, idade, gênero e disparidades raciais
formula = "lsalariohora ~ educ + idade + feminino + preta + parda + amarela + indigena"
mod = ols(formula, df).fit(use_t=True)

## Vendo os resultados
print(mod.summary())

In [ ]:
## Teste t
H0 = "preta = 0"
mod.wald_test(H0)

In [ ]:
## Teste F
H0 = "educ = idade = 0"
mod.wald_test(H0)

In [ ]:
## Modelo mais completo com educ, educ2, idade, idade2, gênero e disparidades raciais
"""
df['educsq'] = df['educ']**2
formula = "lsalariohora ~ educ + educsq + idade + idadesq + feminino + preta + parda + amarela + indigena"
mod = ols(formula, df).fit(use_t=True)

## Vendo os resultados
print(mod.summary())
"""

## Pressupostos do Modelo (Hipóteses de Gauss-Markov)

1. Linearidade nos parâmetros (temos);

2. Amostra aleatória (temos?);

-> Os resultados de MQO podem ser atribuídos para a **amostra**, mas, na PNADC e em outras pesquisas amostrais, deve-se usar pesos para generalizar os resultados

3. Colinearidade não-perfeita (nenhuma variável é função perfeita da outra; temos);

 -> Importância de retirar uma dummy!

4. **MÉDIA CONDICIONAL 0**

Não há **nenhum fator** relegado ao termo de erro que é correlacionado com algumas das variáveis tidas como exógenas (problema clássico: educ e aptidão).
Na prática, essa hipótese é muito forte; em grandes amostras, é possível usar propriedades assintóticas do estimador de MQO e testar a **consistência** do estimador, ou seja, cov(x_j, u) = 0 para todos x_j regressores.

As quatro hipóteses acima garantem a consistência de MQO, ou seja, estimativas centrais precisas.

5. **Homoscedasticidade**

Nome assustador para dizer algo simples: a variância do erro independe das características individuais (sob essa hipótese, pessoas mais ricas tem que ter a mesma variação de aptidão que pessoas com menos renda).

Essa hipótese permite calcular os erros-padrão dos estimadores e encontrar intervalos de confiança estatisticamente corretos para os parâmetros.

As 5 hipóteses acima são as **Hipóteses de Gauss-Markov**

In [ ]:
## Caso o import abaixo dê erro, rodar (demora um pouquinho):
# ! pip install --upgrade Cython
# ! pip install --upgrade git+https://github.com/statsmodels/statsmodels
# import statsmodels.api as sm

In [ ]:
## Testando a hipótese 4: teste RESET
from statsmodels.stats.diagnostic import het_breuschpagan, linear_reset

def reset_ols(formula, data, cov='normal', level=0.05):
    """
    Executes a RESET test for a OLS model specification, where H0: model is well specified
    It is not necessary to assign the function to an object!

    :param formula : patsy formula
    :param data : dataframe
    :param cov : str
        normal: common standard errors
        robust: HC1 standard errors
    :param level : significance level (default 5%)
    """
    ## getting covariance type
    if cov == 'normal':
        cov_type = 'nonrobust'
    else:
        cov_type = 'HC1'

    ## OLS model 
    mod = ols(formula=formula, data=data).fit(use_t=True, cov_type=cov_type)

    ## executing test
    test = linear_reset(mod, power=3, use_f=False, cov_type=cov_type)
    if test.pvalue < level:
        print(f"The test's p-value is equal to {np.around(test.pvalue, 6)} < {level * 100}%")
        print("Therefore, Ho is rejected (the model is badly specified).")
    else:
        print(f"The test's p-value is equal to {np.around(test.pvalue, 6)} > {level * 100}%")
        print("Therefore, Ho is NOT rejected (the model is not badly specified).")

In [ ]:
reset_ols(formula, df, cov='robust')

In [ ]:
## Testando a hipótese 5: breusch-pagan!
from statsmodels.compat import lzip

lNomesBP = ['Multiplicador de Lagrange', 'P-valor', 'Estatística F', 'P-valor F']
lTestBP = het_breuschpagan(mod.resid, mod.model.exog)
lzip(lNomesBP, lTestBP)

# Resultado: com certeza há heteroscedasticidade

## Como consertar heteroscedasticidade?

1. Se você conhecer a forma da heteroscedasticidade (muito difícil)
Casos clássicos: se, ao invés de dados individuais, possuirmos apenas dados médios de algum grupo ou região geográfica; modelo de probabilidade linear (para uma discussão muito boa e inteligível, ver Introdução à Econometria do Wooldridge, Seções 8.4 até o fim do capítulo 8)

2. Usar erros padrão robusto! Em python, eles são tidos como HC1 (iguais à opção *robust* no STATA)